In [26]:
import os
import json
import yaml
import boto3
import logging
from pathlib import Path
from botocore.exceptions import ClientError

In [27]:
# Setup logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [28]:
# global constants
LAMBDA_DIR: str = "lambda"
CONFIG_FILE: str = "config.yml"
CODE_GEN_LAMBDA: str = "code-gen"

In [29]:
config = yaml.safe_load(Path(CONFIG_FILE).read_text())
logger.info(f"config=\n{json.dumps(config, indent=2)}")

[2025-01-18 16:11:42,410] p15196 {3811199622.py:2} INFO - config=
{
  "general": {
    "app_name": "code-gen-agent",
    "role_name": "CodeGenLambdaRole",
    "region": "us-east-1"
  },
  "prompt_info": {
    "name": "USACO_{model_id}",
    "description": "Generate code for the USACO benchmark"
  },
  "prompt_templates": {
    "nova": {
      "models": [
        "amazon.nova-pro-v1:0",
        "amazon.nova-lite-v1:0",
        "amazon.nova-micro-v1:0"
      ],
      "text": "Please reply with a Python 3 solution to the below problem. Read the general instructions below that are applicable to every task,\n\n{{question}}\n\nImportant instructions to follow:\n\n1. Ensure that your code is wrapped in '```python' and '```' Markdown delimiters.\n2. Provide exactly one block of code containing the entire solution.\n3. The code should include a main function and an `if __name__ == \"__main__\"` block as the entry point.\n4. Begin by reasoning through the problem and conceptualizing a solution. 

In [30]:
!source .venv/bin/activate &&  cd {LAMBDA_DIR} && ./build_and_push.sh code_gen && cd -

export REGION=us-east-1
region=us-east-1, image=latest, account=015469603702, image=latest, repo=code_gen#, base image=
"code_gen"
code_gen already exists, no need to create it
user=ubuntu, building the container image 015469603702.dkr.ecr.us-east-1.amazonaws.com/code_gen:latest
DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  15.87kB
Step 1/8 : FROM amazon/aws-lambda-python:3.11
 ---> c14937f0dcb9
Step 2/8 : COPY requirements.txt ${LAMBDA_TASK_ROOT}
 ---> Using cache
 ---> 28f4df09e4e6
Step 3/8 : RUN pip install --upgrade pip
 ---> Using cache
 ---> ea6e3f428dea
Step 4/8 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> a38230ff9935
Step 5/8 : RUN pip install -U boto3 botocore
 ---> Using cache
 ---> be7ad0d6212a
Step 6/8 : COPY index.py ${LAMBDA_TASK_ROOT}
 ---> Using cache
 ---

In [31]:
# create a lambda function using this image
def function_exists(function_name: str, region_name: str) -> bool:
        """Check if Lambda function exists."""
        try:
            client = boto3.client("lambda", region_name=region_name)
            client.get_function(FunctionName=function_name)
            return True
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                return False
            raise

IMAGE_URI_FILE: str = os.path.join(LAMBDA_DIR, "image")
image = Path(IMAGE_URI_FILE).read_text().strip()
role_name: str = config['general']['role_name']
account: str = boto3.client('sts').get_caller_identity()['Account']
role = f"arn:aws:iam::{account}:role/{role_name}"
memory = 1024*10
timeout: int = 90
region_name: str = config['general']['region']
lambda_name: str = CODE_GEN_LAMBDA

client = boto3.client("lambda", region_name=region_name)

if function_exists(lambda_name, region_name) is not True:
    logger.info(f"{lambda_name} lambda does not exist, creating it now")
    logger.info(f"building lambda function with lambda_name={lambda_name}, image={image}, region_name={region_name}, role={role}, memory={memory}, timeout={timeout}")

    response = client.create_function(
        PackageType='Image',  # Specify we're using a container image
        Code={
            'ImageUri': image
        },
        Description='Generate code using Amazon Bedrock',
        Environment={
            'Variables': {
                'DUMMY_KEY': 'VALUE'
            },
        },
        FunctionName=lambda_name,
        MemorySize=memory,
        Publish=True,
        Role=role,
        Tags={
            'TEAM': 'GenAIDevs',
        },
        Timeout=timeout,
        TracingConfig={
            'Mode': 'Active',
        },
        Architectures=['x86_64']  # or ['arm64'] for ARM-based
    )
    logger.info(response)
else:
     logger.info(f"{lambda_name} lambda does exist, updating it with latest code")
     client.update_function_code(
                    FunctionName=lambda_name,
                    ImageUri=image,
                    Publish=True
                )
     

[2025-01-18 16:11:45,176] p15196 {2037351916.py:55} INFO - code-gen lambda does exist, updating it with latest code


In [32]:
logger.info(f"lambda creation or update done")

[2025-01-18 16:11:46,046] p15196 {2846100190.py:1} INFO - lambda creation or update done
